# Implementing the Neural Network and evaluating it's performance on the handwritten digits data set


In [1]:
import numpy as np
import pandas as pd  
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder  
from scipy.optimize import minimize
%matplotlib inline


data = loadmat('ex3data1.mat')  


/home/sravanthi/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


*We have imported the hand-written digits data set. This dataset consists of 5000 instances with each training example of 20X20 (Gray scale format). Hence, the input_size is 400 (20X20). We then split the data into training and testing test using train_test_split method. Please note that we use only the training set for our cross validation (This will be discussed further). Also, this dataset is taken from MNIST dataset*

In [2]:
X = data['X']  
y = data['y']  


X,X_test,y,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

kfl=KFold(n_splits=5, random_state=42, shuffle=True)

X.shape, y.shape, X_test.shape, y_test.shape

((4000, 400), (4000, 1), (1000, 400), (1000, 1))

*Here we used one-hot encoding as it turns a class label 'n' out of k classes into a vector of length k. We used Scikit's library for this.*

In [3]:
encoder = OneHotEncoder(sparse=False)  
y_onehot = encoder.fit_transform(y)  
y_onehot.shape


y_onehot_test=encoder.fit_transform(y_test)
y_onehot_test.shape, y_onehot.shape

((1000, 10), (4000, 10))

In [4]:
y[0], y_onehot[0,:]

(array([8], dtype=uint8),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]))

*Sigmoid Function Implementation: This would bring the prediction values to range of 0 to 1*

In [5]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

*Forward Propagation Implementation: Forward Propagation function calculates the hypothesis for each training instance given the current parameters at each layer in the network. The hypothesis vector 'h', which contains the prediction probabilities for each class, should match our one-hot encoding for y. The cost function then calculates the accuracy of the hypothesis.*

In [6]:
def forward_propagate(X, theta1, theta2):  
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

*Cost Function Implementation: Implements the cost function for determining the error of the neural netwoek predictions. It is required for the competition of accuracy of predictions when we test the network and also for the back propagation algorithm. We need the initial set of parameters as well as input layer size, hidden layer size, no of labels and the learning rate.*

In [7]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):  
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    
    print (h.shape)
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    return J

*The output layer will have 10 units corresponding to the one hot encoding for the class labels.*

In [8]:
# initial setup

input_size = 400 #20*20 grayscale image
hidden_size = 25 # One hidden layer with 25 nodes or units
num_labels = 10
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0] 
print (m)
X = np.matrix(X)  
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

4000


((25, 401), (10, 26))

In [9]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)  
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((4000, 401), (4000, 25), (4000, 26), (4000, 10), (4000, 10))

In [10]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

(4000, 10)


6.7259471099455803

*Next we have to add regularization to the cost function. Before that we have to compute the gradient of the sigmoid function which we implemented earlier. This is the derivative of the sigmoid.*

In [11]:
def sigmoid_gradient(z):  
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

*Back Propagation Function Implementation:*

*In the first half of the function, we are calculating the error by running the data and the current parameters through the "network" (the forward-propagate function) and comparing the output to the true labels. The total error across the whole data set is represented as 'J' and this is the part we discussed earlier as the cost function. After this, we are adjusting the parameters to reduce the error the next time we run through the network. We did this by computing the contributions at each layer to the total error and adjusting appropriately by coming up with a "gradient" matrix (or, how much to change each parameter and in what direction). *

In [12]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):  
    ##### this section is identical to the cost function logic we already saw #####
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    ##### end of cost function logic, below is the new part #####

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d3t = ht - yt  # (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)

        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    delta1 = delta1 / m
    delta2 = delta2 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [13]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)  
J, grad.shape

(6.7326112431925793, (10285,))

*This can be treated as initializing the model. Minimize is an inbuilt scipy method. We train our neural network with the training dataset. Resulting fmin is considered as our model*

In [14]:
# minimize the objective function

#num_labels_2=4
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),  
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 0.37287060496720426
     jac: array([  3.88503121e-04,   2.12856004e-06,  -3.23313603e-06, ...,
        -7.71775952e-04,  -1.06789053e-04,  -8.45805727e-05])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 18
  status: 3
 success: False
       x: array([-1.16207383,  0.00851424, -0.01293254, ...,  0.4938369 ,
       -0.89590773, -1.28918626])

*Here, we obtain our predictions from forward_propagation. This stage is considered as prediction.*

In [15]:
X_test = np.matrix(X_test)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X_test, theta1, theta2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  
y_pred

array([[ 3],
       [ 5],
       [ 5],
       [ 2],
       [ 1],
       [10],
       [ 1],
       [ 4],
       [ 4],
       [ 3],
       [ 4],
       [10],
       [ 1],
       [ 1],
       [ 6],
       [ 7],
       [ 2],
       [ 5],
       [ 3],
       [ 2],
       [ 1],
       [ 6],
       [ 2],
       [ 1],
       [ 9],
       [ 1],
       [ 8],
       [10],
       [ 9],
       [10],
       [ 3],
       [10],
       [ 9],
       [ 9],
       [ 3],
       [ 5],
       [ 7],
       [ 8],
       [ 9],
       [ 4],
       [ 7],
       [ 8],
       [10],
       [10],
       [ 1],
       [ 7],
       [ 7],
       [ 9],
       [ 6],
       [ 5],
       [ 9],
       [ 9],
       [ 4],
       [ 5],
       [ 2],
       [ 5],
       [ 7],
       [10],
       [ 3],
       [ 4],
       [ 4],
       [ 6],
       [ 8],
       [ 7],
       [10],
       [ 5],
       [ 6],
       [ 4],
       [ 7],
       [ 9],
       [ 3],
       [10],
       [10],
       [10],
       [ 2],
       [ 4],
       [ 1],

Calculating the accuracy: 

In [16]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('Accuracy of our implementation without KFold ' +str(accuracy*100)+'%')

Accuracy of our implementation without KFold 93.10000000000001%


*To obtain an optimal performance, we used kfold cross validation. We split our traning set into training and validation set. Number of folds used here is 5. The model is trained during all the 5 folds and the best model is returned, where it is tested with the actual testing dataset and the accuracy is computed.*

In [17]:
def computeNeuralNetsWithKfold(X,y,kfl):
    accuracyArr=[]
    fminArr=[]
    for train,test in kfl.split(X):
        
      #  print (X[train].shape,X[test].shape,y[train].shape,y[test].shape)
        
        X_1,X_test,y_1,y_test= X[train],X[test],y[train],y[test]
       # print (X.shape, X_test.shape, y.shape, y_test.shape)
        
        #print (train, test)
        y_onehot =  encoder.fit_transform(y_1) 
        fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X_1, y_onehot, learning_rate),  
                method='TNC', jac=True, options={'maxiter': 250})
        
        
        fminArr.append(fmin)
        X_test1 = np.matrix(X_test)  
        theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
        theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

        a1, z2, a2, z3, h = forward_propagate(X_test1, theta1, theta2)  
        y_pred = np.array(np.argmax(h, axis=1) + 1)  
        correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]  
        accuracy = (sum(map(int, correct)) / float(len(correct)))
        accuracyArr.append(accuracy)
        print ('Processing Fold '+str(len(accuracyArr)))
    
    bestfmin= fminArr[accuracyArr.index(np.amax(accuracyArr))]
    
    return bestfmin

In [ ]:
#kfl=KFold(n_splits=5, random_state=42, shuffle=True)
fmin=computeNeuralNetsWithKfold(X,y,kfl)
#print (fmin)
print ('Calculated Bestfmin (Best model)')

#This is where the best model is tested with the actual testing dataset.

X_t = np.matrix(X_test)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X_t, theta1, theta2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  

print ('Calculated Y_pred')




Processing Fold 1
Processing Fold 2
Processing Fold 3
Processing Fold 4


In [64]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('Accuracy with Kfold cross validation ' +str(accuracy*100)+'%')

Accuracy with Kfold cross validation 92.7%
